In [1]:
import pycaret
from pycaret.regression import *
from pycaret.classification import *
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
import scipy.stats as stats
from tableone import TableOne

In [ ]:
delta = pd.read_csv(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\FDATA_DELTA.csv", index_col=0) 
current_delta = delta.columns.tolist()
new_delta = {col: 'delta_' + col for col in current_delta}
delta = delta.rename(columns=new_delta)
delta

,delta_exponential_firstorder_10Percentile,delta_exponential_firstorder_90Percentile,delta_exponential_firstorder_Energy,delta_exponential_firstorder_Entropy,delta_exponential_firstorder_InterquartileRange,delta_exponential_firstorder_Kurtosis,delta_exponential_firstorder_Maximum,delta_exponential_firstorder_Mean,delta_exponential_firstorder_MeanAbsoluteDeviation,delta_exponential_firstorder_Median,...,delta_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,0.675148,0.735456,-4140.306429,0.226077,0.035698,17.369658,0.919975,0.711469,0.000902,0.702301,...,-2.939604,0.015000,1.100287,0.002703,-2.702708e+07,6.140874,-3.413193e-03,0.762300,0.001910,0.007340
17815426,-1.378771,-0.812083,23710.359822,-0.287461,0.042827,-11.843144,-0.228153,-0.885610,0.056612,-0.870484,...,-0.346479,-0.078168,2.192016,-0.002891,4.067358e+05,-4.156828,4.915786e-07,-3.938501,-0.014629,-0.004841
20109646,-0.051562,0.070556,-22986.950394,0.029766,0.196582,-0.558412,0.054722,-0.033207,0.012600,-0.106148,...,-0.268550,0.167239,-0.423364,-0.000138,-2.448295e+07,45.593527,-5.579519e-04,0.819366,0.004609,-0.008593
22320836,-0.597642,-0.321917,-32840.788869,-0.067980,-0.010799,-18.551138,-0.561002,-0.366121,0.079080,-0.314536,...,1.833348,-0.012926,-0.787715,0.000246,-1.443771e+07,55.996462,-1.601062e-03,1.278782,0.008829,-0.021723
32221501,-0.401871,-0.450823,4515.451043,-0.496489,0.013552,-0.990929,-0.396543,-0.418202,-0.065599,-0.478765,...,-3.072244,0.029099,0.552622,-0.000590,1.702725e+06,118.833254,-7.873998e-04,-0.576245,0.002344,-0.016453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,-0.812421,-0.929295,13478.760869,-0.085846,-0.026260,-9.568450,-1.564249,-0.871874,-0.046159,-0.887861,...,-1.006784,0.293610,-1.107076,-0.008103,6.644540e+06,8.430760,4.801668e-03,-0.422205,-0.000708,-0.063004
56124435,-0.696750,-0.784821,31115.672140,-0.149014,-0.028701,6.530691,-0.874461,-0.750079,-0.035677,-0.771784,...,0.297115,0.035242,0.106144,-0.001309,1.801669e+07,75.740550,-9.841341e-04,-0.713766,-0.000298,-0.037373
56131318,-0.803772,-0.435870,81776.347946,-0.263599,-0.010126,-13.608512,-0.229142,-0.469046,0.060723,-0.430194,...,-0.404555,-0.000336,-0.272419,-0.003910,6.528883e+07,32.118896,-1.597638e-03,-0.729222,0.001367,-0.053869


In [ ]:
ypCR_data = pd.read_excel(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\clinical_immunologic.xlsx", index_col=0) 
ypCR_data.set_index('pt_no', inplace=True)
ypCR_data = ypCR_data[ypCR_data['Pathol_available'] == 1]
ypCR_data

,name,Ineligible,Ineligible_2019,Pathol_available,Hospital,note,base.fu,date.dx,dob,age,...,note.1,other.malig,pre_PDL1_CPS,pre_PDL1_IC,pre_CD3_num_Pos_mm,pre_CD8_num_Pos_mm,post_PDL1_CPS,post_PDL1_IC,post_CD3_num_Pos_mm,post_CD8_num_Pos_mm
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,정종일,0,0,1.0,1,NaN,2016-11-17,2016-10-24,1946-03-05,69.731507,...,NaN,NaN,5.0,5.0,366.0532,51.9522,10.0,10.0,3171.7243,890.4297
17815426,김진영,1,0,1.0,1,NaN,2021-07-30,2021-07-08,1955-01-04,65.660274,...,NaN,NaN,50.0,40.0,637.6734,374.4912,80.0,80.0,1532.5386,1298.4060
20109646,서대현,0,0,1.0,1,NaN,2018-07-09,2018-05-18,1951-06-20,66.134247,...,NaN,NaN,0.5,0.5,60.6479,39.7833,0.0,0.0,100.5308,43.6339
22320836,최흥우,0,0,1.0,1,NaN,2016-06-07,2016-05-18,1947-07-11,67.961644,...,"diaphragmatic hernia reduction (2020/3/17, 201...",NaN,10.0,10.0,608.9312,140.8317,10.0,10.0,1176.5166,1516.6479
32221501,박남훈,0,0,1.0,1,NaN,2010-10-11,2010-09-17,1939-10-16,70.013699,...,NaN,NaN,1.0,1.0,504.0535,64.6015,10.0,10.0,1245.4099,1155.8683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,최정옥,1,0,1.0,1,Previous malignancy: Breast ca (하지만 1995년이라 ok),2021-10-26,2021-09-29,1961-11-27,59.093151,...,NaN,NaN,10.0,5.0,1152.7963,471.6934,100.0,70.0,1477.2669,868.1072
56124435,김정자,1,0,1.0,1,NaN,2021-12-03,2021-11-09,1963-01-13,58.082192,...,NaN,NaN,0.0,0.0,145.0536,31.3657,10.0,10.0,479.3238,389.0902
56131318,홍성배,1,0,1.0,1,NaN,2021-11-23,2021-11-10,1951-05-20,69.542466,...,NaN,NaN,10.0,10.0,5283.4120,5773.6300,50.0,50.0,4583.2514,4448.2354


In [ ]:
delta['post_PDL1_CPS'] = ypCR_data['post_PDL1_CPS']
delta

,delta_exponential_firstorder_10Percentile,delta_exponential_firstorder_90Percentile,delta_exponential_firstorder_Energy,delta_exponential_firstorder_Entropy,delta_exponential_firstorder_InterquartileRange,delta_exponential_firstorder_Kurtosis,delta_exponential_firstorder_Maximum,delta_exponential_firstorder_Mean,delta_exponential_firstorder_MeanAbsoluteDeviation,delta_exponential_firstorder_Median,...,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength,post_PDL1_CPS
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,0.675148,0.735456,-4140.306429,0.226077,0.035698,17.369658,0.919975,0.711469,0.000902,0.702301,...,0.015000,1.100287,0.002703,-2.702708e+07,6.140874,-3.413193e-03,0.762300,0.001910,0.007340,10.0
17815426,-1.378771,-0.812083,23710.359822,-0.287461,0.042827,-11.843144,-0.228153,-0.885610,0.056612,-0.870484,...,-0.078168,2.192016,-0.002891,4.067358e+05,-4.156828,4.915786e-07,-3.938501,-0.014629,-0.004841,80.0
20109646,-0.051562,0.070556,-22986.950394,0.029766,0.196582,-0.558412,0.054722,-0.033207,0.012600,-0.106148,...,0.167239,-0.423364,-0.000138,-2.448295e+07,45.593527,-5.579519e-04,0.819366,0.004609,-0.008593,0.0
22320836,-0.597642,-0.321917,-32840.788869,-0.067980,-0.010799,-18.551138,-0.561002,-0.366121,0.079080,-0.314536,...,-0.012926,-0.787715,0.000246,-1.443771e+07,55.996462,-1.601062e-03,1.278782,0.008829,-0.021723,10.0
32221501,-0.401871,-0.450823,4515.451043,-0.496489,0.013552,-0.990929,-0.396543,-0.418202,-0.065599,-0.478765,...,0.029099,0.552622,-0.000590,1.702725e+06,118.833254,-7.873998e-04,-0.576245,0.002344,-0.016453,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,-0.812421,-0.929295,13478.760869,-0.085846,-0.026260,-9.568450,-1.564249,-0.871874,-0.046159,-0.887861,...,0.293610,-1.107076,-0.008103,6.644540e+06,8.430760,4.801668e-03,-0.422205,-0.000708,-0.063004,100.0
56124435,-0.696750,-0.784821,31115.672140,-0.149014,-0.028701,6.530691,-0.874461,-0.750079,-0.035677,-0.771784,...,0.035242,0.106144,-0.001309,1.801669e+07,75.740550,-9.841341e-04,-0.713766,-0.000298,-0.037373,10.0
56131318,-0.803772,-0.435870,81776.347946,-0.263599,-0.010126,-13.608512,-0.229142,-0.469046,0.060723,-0.430194,...,-0.000336,-0.272419,-0.003910,6.528883e+07,32.118896,-1.597638e-03,-0.729222,0.001367,-0.053869,50.0


In [ ]:
# bin_labels = [0, 1, 2]
# bins = [-1, 1, 10, float('inf')]
# post['encoded_PDL1'] = pd.cut(post['post_PDL1_CPS'], bins=bins, labels=bin_labels)
# post
conditions = [
    (delta['post_PDL1_CPS'] >= 0) & (delta['post_PDL1_CPS'] < 1),
    (delta['post_PDL1_CPS'] >= 1) & (delta['post_PDL1_CPS'] <= 10),
    (delta['post_PDL1_CPS'] > 10)
]
choices = [0, 1, 2]
delta['encoded_PDL1'] = np.select(conditions, choices, default=np.nan)
delta

,delta_exponential_firstorder_10Percentile,delta_exponential_firstorder_90Percentile,delta_exponential_firstorder_Energy,delta_exponential_firstorder_Entropy,delta_exponential_firstorder_InterquartileRange,delta_exponential_firstorder_Kurtosis,delta_exponential_firstorder_Maximum,delta_exponential_firstorder_Mean,delta_exponential_firstorder_MeanAbsoluteDeviation,delta_exponential_firstorder_Median,...,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength,post_PDL1_CPS,encoded_PDL1
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,0.675148,0.735456,-4140.306429,0.226077,0.035698,17.369658,0.919975,0.711469,0.000902,0.702301,...,1.100287,0.002703,-2.702708e+07,6.140874,-3.413193e-03,0.762300,0.001910,0.007340,10.0,1.0
17815426,-1.378771,-0.812083,23710.359822,-0.287461,0.042827,-11.843144,-0.228153,-0.885610,0.056612,-0.870484,...,2.192016,-0.002891,4.067358e+05,-4.156828,4.915786e-07,-3.938501,-0.014629,-0.004841,80.0,2.0
20109646,-0.051562,0.070556,-22986.950394,0.029766,0.196582,-0.558412,0.054722,-0.033207,0.012600,-0.106148,...,-0.423364,-0.000138,-2.448295e+07,45.593527,-5.579519e-04,0.819366,0.004609,-0.008593,0.0,0.0
22320836,-0.597642,-0.321917,-32840.788869,-0.067980,-0.010799,-18.551138,-0.561002,-0.366121,0.079080,-0.314536,...,-0.787715,0.000246,-1.443771e+07,55.996462,-1.601062e-03,1.278782,0.008829,-0.021723,10.0,1.0
32221501,-0.401871,-0.450823,4515.451043,-0.496489,0.013552,-0.990929,-0.396543,-0.418202,-0.065599,-0.478765,...,0.552622,-0.000590,1.702725e+06,118.833254,-7.873998e-04,-0.576245,0.002344,-0.016453,10.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,-0.812421,-0.929295,13478.760869,-0.085846,-0.026260,-9.568450,-1.564249,-0.871874,-0.046159,-0.887861,...,-1.107076,-0.008103,6.644540e+06,8.430760,4.801668e-03,-0.422205,-0.000708,-0.063004,100.0,2.0
56124435,-0.696750,-0.784821,31115.672140,-0.149014,-0.028701,6.530691,-0.874461,-0.750079,-0.035677,-0.771784,...,0.106144,-0.001309,1.801669e+07,75.740550,-9.841341e-04,-0.713766,-0.000298,-0.037373,10.0,1.0
56131318,-0.803772,-0.435870,81776.347946,-0.263599,-0.010126,-13.608512,-0.229142,-0.469046,0.060723,-0.430194,...,-0.272419,-0.003910,6.528883e+07,32.118896,-1.597638e-03,-0.729222,0.001367,-0.053869,50.0,2.0


In [ ]:
delta.drop(columns=['post_PDL1_CPS'], inplace=True)
delta

,delta_exponential_firstorder_10Percentile,delta_exponential_firstorder_90Percentile,delta_exponential_firstorder_Energy,delta_exponential_firstorder_Entropy,delta_exponential_firstorder_InterquartileRange,delta_exponential_firstorder_Kurtosis,delta_exponential_firstorder_Maximum,delta_exponential_firstorder_Mean,delta_exponential_firstorder_MeanAbsoluteDeviation,delta_exponential_firstorder_Median,...,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength,encoded_PDL1
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,0.675148,0.735456,-4140.306429,0.226077,0.035698,17.369658,0.919975,0.711469,0.000902,0.702301,...,0.015000,1.100287,0.002703,-2.702708e+07,6.140874,-3.413193e-03,0.762300,0.001910,0.007340,1.0
17815426,-1.378771,-0.812083,23710.359822,-0.287461,0.042827,-11.843144,-0.228153,-0.885610,0.056612,-0.870484,...,-0.078168,2.192016,-0.002891,4.067358e+05,-4.156828,4.915786e-07,-3.938501,-0.014629,-0.004841,2.0
20109646,-0.051562,0.070556,-22986.950394,0.029766,0.196582,-0.558412,0.054722,-0.033207,0.012600,-0.106148,...,0.167239,-0.423364,-0.000138,-2.448295e+07,45.593527,-5.579519e-04,0.819366,0.004609,-0.008593,0.0
22320836,-0.597642,-0.321917,-32840.788869,-0.067980,-0.010799,-18.551138,-0.561002,-0.366121,0.079080,-0.314536,...,-0.012926,-0.787715,0.000246,-1.443771e+07,55.996462,-1.601062e-03,1.278782,0.008829,-0.021723,1.0
32221501,-0.401871,-0.450823,4515.451043,-0.496489,0.013552,-0.990929,-0.396543,-0.418202,-0.065599,-0.478765,...,0.029099,0.552622,-0.000590,1.702725e+06,118.833254,-7.873998e-04,-0.576245,0.002344,-0.016453,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,-0.812421,-0.929295,13478.760869,-0.085846,-0.026260,-9.568450,-1.564249,-0.871874,-0.046159,-0.887861,...,0.293610,-1.107076,-0.008103,6.644540e+06,8.430760,4.801668e-03,-0.422205,-0.000708,-0.063004,2.0
56124435,-0.696750,-0.784821,31115.672140,-0.149014,-0.028701,6.530691,-0.874461,-0.750079,-0.035677,-0.771784,...,0.035242,0.106144,-0.001309,1.801669e+07,75.740550,-9.841341e-04,-0.713766,-0.000298,-0.037373,1.0
56131318,-0.803772,-0.435870,81776.347946,-0.263599,-0.010126,-13.608512,-0.229142,-0.469046,0.060723,-0.430194,...,-0.000336,-0.272419,-0.003910,6.528883e+07,32.118896,-1.597638e-03,-0.729222,0.001367,-0.053869,2.0


In [ ]:
feature = delta.drop(columns='encoded_PDL1')
target = delta['encoded_PDL1']
list_numerical = feature.columns.to_list()
list_numerical

['delta_exponential_firstorder_10Percentile',
 'delta_exponential_firstorder_90Percentile',
 'delta_exponential_firstorder_Energy',
 'delta_exponential_firstorder_Entropy',
 'delta_exponential_firstorder_InterquartileRange',
 'delta_exponential_firstorder_Kurtosis',
 'delta_exponential_firstorder_Maximum',
 'delta_exponential_firstorder_Mean',
 'delta_exponential_firstorder_MeanAbsoluteDeviation',
 'delta_exponential_firstorder_Median',
 'delta_exponential_firstorder_Minimum',
 'delta_exponential_firstorder_Range',
 'delta_exponential_firstorder_RobustMeanAbsoluteDeviation',
 'delta_exponential_firstorder_RootMeanSquared',
 'delta_exponential_firstorder_Skewness',
 'delta_exponential_firstorder_TotalEnergy',
 'delta_exponential_firstorder_Uniformity',
 'delta_exponential_firstorder_Variance',
 'delta_exponential_glcm_Autocorrelation',
 'delta_exponential_glcm_ClusterProminence',
 'delta_exponential_glcm_ClusterShade',
 'delta_exponential_glcm_ClusterTendency',
 'delta_exponential_glcm_

In [ ]:
feature

,delta_exponential_firstorder_10Percentile,delta_exponential_firstorder_90Percentile,delta_exponential_firstorder_Energy,delta_exponential_firstorder_Entropy,delta_exponential_firstorder_InterquartileRange,delta_exponential_firstorder_Kurtosis,delta_exponential_firstorder_Maximum,delta_exponential_firstorder_Mean,delta_exponential_firstorder_MeanAbsoluteDeviation,delta_exponential_firstorder_Median,...,delta_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,0.675148,0.735456,-4140.306429,0.226077,0.035698,17.369658,0.919975,0.711469,0.000902,0.702301,...,-2.939604,0.015000,1.100287,0.002703,-2.702708e+07,6.140874,-3.413193e-03,0.762300,0.001910,0.007340
17815426,-1.378771,-0.812083,23710.359822,-0.287461,0.042827,-11.843144,-0.228153,-0.885610,0.056612,-0.870484,...,-0.346479,-0.078168,2.192016,-0.002891,4.067358e+05,-4.156828,4.915786e-07,-3.938501,-0.014629,-0.004841
20109646,-0.051562,0.070556,-22986.950394,0.029766,0.196582,-0.558412,0.054722,-0.033207,0.012600,-0.106148,...,-0.268550,0.167239,-0.423364,-0.000138,-2.448295e+07,45.593527,-5.579519e-04,0.819366,0.004609,-0.008593
22320836,-0.597642,-0.321917,-32840.788869,-0.067980,-0.010799,-18.551138,-0.561002,-0.366121,0.079080,-0.314536,...,1.833348,-0.012926,-0.787715,0.000246,-1.443771e+07,55.996462,-1.601062e-03,1.278782,0.008829,-0.021723
32221501,-0.401871,-0.450823,4515.451043,-0.496489,0.013552,-0.990929,-0.396543,-0.418202,-0.065599,-0.478765,...,-3.072244,0.029099,0.552622,-0.000590,1.702725e+06,118.833254,-7.873998e-04,-0.576245,0.002344,-0.016453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,-0.812421,-0.929295,13478.760869,-0.085846,-0.026260,-9.568450,-1.564249,-0.871874,-0.046159,-0.887861,...,-1.006784,0.293610,-1.107076,-0.008103,6.644540e+06,8.430760,4.801668e-03,-0.422205,-0.000708,-0.063004
56124435,-0.696750,-0.784821,31115.672140,-0.149014,-0.028701,6.530691,-0.874461,-0.750079,-0.035677,-0.771784,...,0.297115,0.035242,0.106144,-0.001309,1.801669e+07,75.740550,-9.841341e-04,-0.713766,-0.000298,-0.037373
56131318,-0.803772,-0.435870,81776.347946,-0.263599,-0.010126,-13.608512,-0.229142,-0.469046,0.060723,-0.430194,...,-0.404555,-0.000336,-0.272419,-0.003910,6.528883e+07,32.118896,-1.597638e-03,-0.729222,0.001367,-0.053869


In [ ]:
# def custom_zscore(column):
#     if column.std() == 0:
#         return (column - column.mean())  # For columns with zero variance, mean-centered values
#     else:
#         return (column - column.mean()) / column.std()

# # Apply custom_zscore to each column
# feature = feature.apply(custom_zscore)
# feature

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import Lasso, LassoCV
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [ ]:
# lasso 를 위한 데이터 나누기 
X_train, X_test, y_train, y_test = train_test_split(feature, target, shuffle=True, test_size = 0.2, stratify=target, random_state=2023)

In [ ]:
X_train

,delta_exponential_firstorder_10Percentile,delta_exponential_firstorder_90Percentile,delta_exponential_firstorder_Energy,delta_exponential_firstorder_Entropy,delta_exponential_firstorder_InterquartileRange,delta_exponential_firstorder_Kurtosis,delta_exponential_firstorder_Maximum,delta_exponential_firstorder_Mean,delta_exponential_firstorder_MeanAbsoluteDeviation,delta_exponential_firstorder_Median,...,delta_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
41375060,-0.346235,-0.285526,7417.454609,-0.013437,0.052366,-16.901355,-0.105700,-0.317725,0.001427,-0.345913,...,3.712660,0.045608,1.525231,0.008755,-3.789789e+06,44.931031,-0.007340,0.149927,0.000608,-0.043247
37567637,-0.482967,-0.602744,19512.482757,-0.847293,-0.048396,69.752300,-0.001301,-0.538524,-0.062427,-0.573567,...,-2.767768,0.218069,-0.639354,-0.008054,1.952179e+07,-7.709710,0.020640,-1.845684,-0.011488,0.005915
49087653,-0.177715,-0.246449,24421.991508,0.000000,-0.026035,33.942890,-0.278884,-0.208867,-0.035619,-0.228895,...,-2.105801,-0.015490,-0.851733,-0.002741,3.583023e+07,-10.389029,0.004871,-2.040553,-0.006333,0.051593
46695156,-0.400164,0.110858,19058.311722,-0.188515,0.110325,-259.982979,-9.075972,-0.079407,0.072918,0.033427,...,-13.651342,-0.014117,-1.188595,-0.010801,2.913572e+06,19.467073,-0.007118,-6.084096,0.011340,-1.422193
54502130,-0.745145,-0.682332,27741.308482,-0.049317,0.005894,-13.794402,-0.028265,-0.723429,0.001705,-0.754327,...,4.448422,-0.009817,-0.764656,-0.002872,5.336440e+06,36.094545,-0.013138,2.579153,0.002983,-0.164958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47047248,-0.508550,-0.595669,11525.265016,0.557829,0.010509,-2.135980,-1.549718,-0.582763,-0.018458,-0.663227,...,-3.931039,-0.116855,-0.175324,-0.002942,8.552021e+05,62.221669,-0.000776,-1.385447,0.009070,-0.024865
37509338,-0.703671,-0.712051,7927.791933,-0.084262,-0.019409,10.925781,-0.817454,-0.717087,0.013736,-0.698815,...,0.639735,0.010700,3.000000,0.000345,3.086606e+06,6.305412,-999999.990676,0.292578,0.000762,0.048651
50740947,-0.220197,-0.193698,20041.304810,-0.032400,0.009768,-68.871905,0.071302,-0.216302,0.024254,-0.205515,...,-2.627177,-0.051192,1.712897,-0.000592,1.976091e+07,7.873596,-0.012838,0.428536,0.000517,-0.141541


In [ ]:
X_test

,delta_exponential_firstorder_10Percentile,delta_exponential_firstorder_90Percentile,delta_exponential_firstorder_Energy,delta_exponential_firstorder_Entropy,delta_exponential_firstorder_InterquartileRange,delta_exponential_firstorder_Kurtosis,delta_exponential_firstorder_Maximum,delta_exponential_firstorder_Mean,delta_exponential_firstorder_MeanAbsoluteDeviation,delta_exponential_firstorder_Median,...,delta_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
42748515,-0.480986,-0.375686,-3515.728953,-0.216486,0.045160,-29.290245,-0.228123,-0.441975,0.053402,-0.419438,...,5.961157,0.018577,1.017141,0.003584,-2.221883e+06,-51.965991,0.000283,-0.162587,-0.005935,0.017544
22320836,-0.597642,-0.321917,-32840.788869,-0.067980,-0.010799,-18.551138,-0.561002,-0.366121,0.079080,-0.314536,...,1.833348,-0.012926,-0.787715,0.000246,-1.443771e+07,55.996462,-0.001601,1.278782,0.008829,-0.021723
56131813,-0.690758,-0.710463,54079.416361,-0.106082,-0.003531,-7.307482,-1.479485,-0.702779,-0.010187,-0.707339,...,-0.082819,0.011403,-0.241786,-0.003475,1.278301e+07,229.369008,-0.005528,1.112013,0.008574,-0.042068
37244770,-0.162339,-0.160024,-35012.311274,-0.005243,0.001248,38.207556,-0.116392,-0.174395,0.011459,-0.169024,...,0.922495,-0.518547,1.870035,0.000564,-4.186273e+07,0.184928,-0.013964,0.359789,0.000599,0.021699
49344356,-0.505061,-0.108589,903.257941,-0.002364,0.016009,-6.085961,-0.496869,-0.154260,0.066203,-0.110069,...,4.413640,-0.007191,-0.055995,0.000013,5.950454e+05,9.670855,-0.000673,0.152857,0.001436,-0.009085
42120720,-0.954672,-0.761824,28578.905591,-0.437230,-0.042092,-5.622064,-1.085083,-0.746830,-0.013539,-0.762118,...,-5.945015,-0.025132,0.873087,-0.007040,7.828450e+06,13.235655,-0.001086,-5.639784,-0.011643,-0.070389
51524302,-0.732591,-0.798539,-9376.894242,-0.035808,-0.031538,21.626052,-1.724830,-0.770567,-0.021255,-0.775383,...,0.659891,-0.047359,0.520184,0.000124,3.477882e+06,11.190002,-0.013543,0.144951,0.000093,-0.062825
47451087,0.052934,-0.014959,18172.233460,0.000000,-0.020095,25.485543,0.094880,0.009051,-0.027111,-0.008526,...,-2.242555,0.214013,0.691563,0.000913,6.537215e+05,15.180199,0.000160,-0.369953,-0.001383,-0.016435
53004114,-0.548635,-0.548991,30701.590584,-0.136401,-0.019871,18.084386,0.833358,-0.578110,-0.002078,-0.585262,...,3.255458,-0.092059,1.468045,-0.000285,1.216261e+06,29.426239,-0.026447,3.760034,0.002288,-0.298484


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train[list_numerical]) 

X_train[list_numerical] = scaler.transform(X_train[list_numerical])

X_test[list_numerical] = scaler.transform(X_test[list_numerical])

In [ ]:
X_train

,delta_exponential_firstorder_10Percentile,delta_exponential_firstorder_90Percentile,delta_exponential_firstorder_Energy,delta_exponential_firstorder_Entropy,delta_exponential_firstorder_InterquartileRange,delta_exponential_firstorder_Kurtosis,delta_exponential_firstorder_Maximum,delta_exponential_firstorder_Mean,delta_exponential_firstorder_MeanAbsoluteDeviation,delta_exponential_firstorder_Median,...,delta_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
41375060,0.480183,0.874858,-0.154018,0.776522,0.549611,-0.019401,0.535855,0.798853,0.299357,0.726231,...,1.041185,0.430590,1.417986,2.970664,-0.773458,0.121528,0.112509,0.284331,0.144845,0.095695
37567637,0.083773,-0.090126,0.424647,-1.769290,-0.159209,0.794878,0.602562,0.035774,-0.469582,-0.049131,...,-0.729295,1.800501,-0.849912,-1.609575,1.157863,0.044854,0.112509,-0.674535,-1.007998,0.341880
49087653,0.968752,0.993732,0.659534,0.817545,-0.001909,0.458379,0.425197,1.175066,-0.146759,1.124781,...,-0.548443,-0.054723,-1.072427,-0.161731,2.508987,0.040952,0.112509,-0.768167,-0.516646,0.570617
46695156,0.323834,2.080665,0.402918,0.241997,0.957330,-2.303623,-5.195789,1.622480,1.160261,2.018217,...,-3.702733,-0.043819,-1.425367,-2.357935,-0.218097,0.084438,0.112509,-2.711037,1.167611,-6.809536
54502130,-0.676326,-0.332234,0.818340,0.666977,0.222703,0.009795,0.585333,-0.603259,0.302700,-0.664775,...,1.242198,-0.009661,-0.981194,-0.197446,-0.017367,0.108657,0.112509,1.451543,0.371115,-0.513785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47047248,0.009603,-0.068603,0.042512,2.520628,0.255168,0.119349,-0.386815,-0.117115,0.059895,-0.354499,...,-1.047106,-0.859898,-0.363735,-0.216729,-0.388629,0.146713,0.112509,-0.453397,0.951292,0.187747
37509338,-0.556085,-0.422641,-0.129602,0.560287,0.044708,0.242089,0.081073,-0.581340,0.447584,-0.475709,...,0.201649,0.153311,2.963144,0.679053,-0.203761,0.065268,-8.888194,0.352873,0.159518,0.555886
50740947,0.845591,1.154199,0.449947,0.718625,0.249951,-0.507764,0.648952,1.149370,0.574243,1.204411,...,-0.690885,-0.338318,1.614610,0.423637,1.177674,0.067552,0.112509,0.418199,0.136157,-0.396522


In [ ]:
n_alphas = 100
alphas = np.logspace(-4, -1.2, n_alphas)
print(alphas)
lasso_cv = LassoCV(alphas = alphas, random_state=2023, max_iter=1000000, cv=KFold(n_splits=5), verbose=True, n_jobs=-1).fit(X_train, y_train)

[0.0001     0.00010673 0.00011391 0.00012158 0.00012976 0.00013849
 0.00014781 0.00015775 0.00016837 0.0001797  0.00019179 0.0002047
 0.00021847 0.00023317 0.00024886 0.00026561 0.00028348 0.00030256
 0.00032292 0.00034465 0.00036784 0.00039259 0.00041901 0.0004472
 0.0004773  0.00050941 0.00054369 0.00058028 0.00061933 0.000661
 0.00070548 0.00075295 0.00080362 0.0008577  0.00091541 0.00097701
 0.00104275 0.00111292 0.00118781 0.00126774 0.00135305 0.0014441
 0.00154127 0.00164498 0.00175568 0.00187382 0.00199991 0.00213448
 0.00227812 0.00243141 0.00259502 0.00276965 0.00295602 0.00315493
 0.00336723 0.00359381 0.00383564 0.00409375 0.00436922 0.00466323
 0.00497702 0.00531193 0.00566938 0.00605088 0.00645804 0.00689261
 0.00735642 0.00785144 0.00837977 0.00894366 0.00954548 0.01018781
 0.01087336 0.01160504 0.01238595 0.01321941 0.01410896 0.01505836
 0.01607166 0.01715313 0.01830738 0.0195393  0.02085412 0.02225742
 0.02375514 0.02535364 0.02705972 0.02888059 0.03082399 0.03289817


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.9s finished


In [ ]:
print('R squared training set', round(lasso_cv.score(X_train, y_train)*100, 2))
print('R squared test set', round(lasso_cv.score(X_test, y_test)*100, 2))

R squared training set 74.83
R squared test set 3.06


In [ ]:
best_alpha = lasso_cv.alpha_
print("Best alpha parameter:", best_alpha)

Best alpha parameter: 0.06309573444801933


In [ ]:
feature_list = lasso_cv.feature_names_in_[lasso_cv.coef_ != 0]
print(len(feature_list), feature_list)

34 ['delta_log-sigma-1-0-mm-3D_gldm_DependenceVariance'
 'delta_log-sigma-1-0-mm-3D_glszm_LargeAreaLowGrayLevelEmphasis'
 'delta_log-sigma-1-0-mm-3D_glszm_ZoneEntropy'
 'delta_log-sigma-1-0-mm-3D_glszm_ZonePercentage'
 'delta_log-sigma-1-5-mm-3D_firstorder_Energy'
 'delta_log-sigma-1-5-mm-3D_firstorder_TotalEnergy'
 'delta_log-sigma-1-5-mm-3D_gldm_DependenceVariance'
 'delta_log-sigma-2-0-mm-3D_ngtdm_Busyness'
 'delta_log-sigma-2-5-mm-3D_glszm_SizeZoneNonUniformity'
 'delta_log-sigma-2-5-mm-3D_glszm_ZoneEntropy'
 'delta_logarithm_glszm_LargeAreaHighGrayLevelEmphasis'
 'delta_original_glszm_ZoneVariance'
 'delta_original_shape_LeastAxisLength' 'delta_square_glcm_Correlation'
 'delta_square_glszm_LowGrayLevelZoneEmphasis'
 'delta_square_glszm_ZonePercentage' 'delta_wavelet-HHH_glszm_ZoneEntropy'
 'delta_wavelet-HHL_firstorder_Median'
 'delta_wavelet-HHL_firstorder_Uniformity'
 'delta_wavelet-HLH_glcm_ClusterShade'
 'delta_wavelet-HLH_glcm_MaximumProbability'
 'delta_wavelet-HLH_glcm_SumE

In [ ]:
lasso_data = feature[feature_list]
lasso_data

,delta_log-sigma-1-0-mm-3D_gldm_DependenceVariance,delta_log-sigma-1-0-mm-3D_glszm_LargeAreaLowGrayLevelEmphasis,delta_log-sigma-1-0-mm-3D_glszm_ZoneEntropy,delta_log-sigma-1-0-mm-3D_glszm_ZonePercentage,delta_log-sigma-1-5-mm-3D_firstorder_Energy,delta_log-sigma-1-5-mm-3D_firstorder_TotalEnergy,delta_log-sigma-1-5-mm-3D_gldm_DependenceVariance,delta_log-sigma-2-0-mm-3D_ngtdm_Busyness,delta_log-sigma-2-5-mm-3D_glszm_SizeZoneNonUniformity,delta_log-sigma-2-5-mm-3D_glszm_ZoneEntropy,...,delta_wavelet-HLL_firstorder_TotalEnergy,delta_wavelet-LHH_glcm_Autocorrelation,delta_wavelet-LHH_glcm_JointEntropy,delta_wavelet-LHH_glrlm_GrayLevelNonUniformityNormalized,delta_wavelet-LHH_glszm_HighGrayLevelZoneEmphasis,delta_wavelet-LHH_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LHL_glszm_SizeZoneNonUniformityNormalized,delta_wavelet-LLH_glszm_GrayLevelNonUniformityNormalized,delta_wavelet-LLH_glszm_SmallAreaEmphasis,delta_wavelet-LLL_glszm_SizeZoneNonUniformityNormalized
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,0.558076,-2.933690e+06,-0.580241,0.000049,-114.443925,-114.443925,-0.279418,-1056.534360,-0.400000,0.078072,...,-197.647973,0.026717,-0.001730,1.103154e-05,0.375000,-0.357639,-0.015918,0.007812,0.217912,-0.063673
17815426,-12.343727,-1.706221e+05,2.869618,0.002286,227.214915,227.214915,-7.075172,1631.058053,-0.022222,2.081036,...,3219.113800,-0.026725,-0.009022,5.173116e-05,0.701613,0.016859,-0.238098,-0.002484,-0.252679,-0.124241
20109646,4.751498,-1.334454e+07,0.350480,0.000185,-297.388370,-297.388370,2.573524,-565.609472,-1.052632,0.355520,...,-2598.354714,0.006331,-0.005611,-5.672285e-07,0.133333,-0.132870,0.067268,-0.004853,-0.059095,0.044766
22320836,6.531945,-8.378362e+06,0.885799,0.000435,1020.117676,1020.117676,1.967860,-2020.429064,-0.909091,0.436802,...,-1988.693499,-0.026800,-0.005635,-6.041939e-07,-0.166667,0.340278,0.140000,0.019497,-0.040512,0.038058
32221501,-4.774107,1.226175e+06,0.412932,-0.000303,-52.620489,-52.620489,-5.953415,6753.488998,0.857143,0.739900,...,-695.601258,-0.008834,-0.015482,-2.838373e-06,0.750000,-0.260417,0.193333,0.007982,-0.178836,0.057469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,-3.462614,2.229554e+06,-0.211914,-0.004948,8.082049,8.082049,-1.749220,1366.838287,0.923077,1.334679,...,-169.814090,-0.014890,-0.010633,-2.478449e-05,0.750000,0.072916,0.160000,0.018409,0.092998,0.065063
56124435,-1.862425,1.000684e+07,0.672799,-0.000162,-99.025895,-99.025895,-0.156764,5301.913800,3.381381,1.126183,...,-433.785447,0.003013,0.001943,-1.121972e-05,-0.750000,0.359375,0.235186,0.072068,0.087433,0.005844
56131318,-1.052585,1.533579e+07,0.253621,-0.001146,1144.163491,1144.163491,-2.477782,79887.975955,3.733333,0.689246,...,3111.624309,0.000017,0.002660,1.520622e-05,0.666667,-0.075256,0.052575,-0.000907,0.144587,0.003868


In [ ]:
final_data = pd.concat([lasso_data, target], axis=1)
final_data

,delta_log-sigma-1-0-mm-3D_gldm_DependenceVariance,delta_log-sigma-1-0-mm-3D_glszm_LargeAreaLowGrayLevelEmphasis,delta_log-sigma-1-0-mm-3D_glszm_ZoneEntropy,delta_log-sigma-1-0-mm-3D_glszm_ZonePercentage,delta_log-sigma-1-5-mm-3D_firstorder_Energy,delta_log-sigma-1-5-mm-3D_firstorder_TotalEnergy,delta_log-sigma-1-5-mm-3D_gldm_DependenceVariance,delta_log-sigma-2-0-mm-3D_ngtdm_Busyness,delta_log-sigma-2-5-mm-3D_glszm_SizeZoneNonUniformity,delta_log-sigma-2-5-mm-3D_glszm_ZoneEntropy,...,delta_wavelet-LHH_glcm_Autocorrelation,delta_wavelet-LHH_glcm_JointEntropy,delta_wavelet-LHH_glrlm_GrayLevelNonUniformityNormalized,delta_wavelet-LHH_glszm_HighGrayLevelZoneEmphasis,delta_wavelet-LHH_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LHL_glszm_SizeZoneNonUniformityNormalized,delta_wavelet-LLH_glszm_GrayLevelNonUniformityNormalized,delta_wavelet-LLH_glszm_SmallAreaEmphasis,delta_wavelet-LLL_glszm_SizeZoneNonUniformityNormalized,encoded_PDL1
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,0.558076,-2.933690e+06,-0.580241,0.000049,-114.443925,-114.443925,-0.279418,-1056.534360,-0.400000,0.078072,...,0.026717,-0.001730,1.103154e-05,0.375000,-0.357639,-0.015918,0.007812,0.217912,-0.063673,1.0
17815426,-12.343727,-1.706221e+05,2.869618,0.002286,227.214915,227.214915,-7.075172,1631.058053,-0.022222,2.081036,...,-0.026725,-0.009022,5.173116e-05,0.701613,0.016859,-0.238098,-0.002484,-0.252679,-0.124241,2.0
20109646,4.751498,-1.334454e+07,0.350480,0.000185,-297.388370,-297.388370,2.573524,-565.609472,-1.052632,0.355520,...,0.006331,-0.005611,-5.672285e-07,0.133333,-0.132870,0.067268,-0.004853,-0.059095,0.044766,0.0
22320836,6.531945,-8.378362e+06,0.885799,0.000435,1020.117676,1020.117676,1.967860,-2020.429064,-0.909091,0.436802,...,-0.026800,-0.005635,-6.041939e-07,-0.166667,0.340278,0.140000,0.019497,-0.040512,0.038058,1.0
32221501,-4.774107,1.226175e+06,0.412932,-0.000303,-52.620489,-52.620489,-5.953415,6753.488998,0.857143,0.739900,...,-0.008834,-0.015482,-2.838373e-06,0.750000,-0.260417,0.193333,0.007982,-0.178836,0.057469,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,-3.462614,2.229554e+06,-0.211914,-0.004948,8.082049,8.082049,-1.749220,1366.838287,0.923077,1.334679,...,-0.014890,-0.010633,-2.478449e-05,0.750000,0.072916,0.160000,0.018409,0.092998,0.065063,2.0
56124435,-1.862425,1.000684e+07,0.672799,-0.000162,-99.025895,-99.025895,-0.156764,5301.913800,3.381381,1.126183,...,0.003013,0.001943,-1.121972e-05,-0.750000,0.359375,0.235186,0.072068,0.087433,0.005844,1.0
56131318,-1.052585,1.533579e+07,0.253621,-0.001146,1144.163491,1144.163491,-2.477782,79887.975955,3.733333,0.689246,...,0.000017,0.002660,1.520622e-05,0.666667,-0.075256,0.052575,-0.000907,0.144587,0.003868,2.0


In [ ]:
clinical_train = pd.read_csv(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\2FDATA_clinical_train.csv", index_col=0) 
clinical_test = pd.read_csv(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\2FDATA_clinical_test.csv", index_col=0) 
clinical_train.drop(columns=['TD', 'RT.tech_IMRT', 'location'], inplace=True)
clinical_test.drop(columns=['TD', 'RT.tech_IMRT', 'location'], inplace=True)
clinical_train

,encoded_PDL1
pt_no,
51127213,1.0
38924846,0.0
56241796,1.0
40815882,1.0
33908849,0.0
...,...
47181472,2.0
49087653,1.0
49588132,0.0


In [ ]:
rad_features_list = final_data.columns.to_list()
rad_features_list.remove('encoded_PDL1')
rad_features_list

['delta_log-sigma-1-0-mm-3D_gldm_DependenceVariance',
 'delta_log-sigma-1-0-mm-3D_glszm_LargeAreaLowGrayLevelEmphasis',
 'delta_log-sigma-1-0-mm-3D_glszm_ZoneEntropy',
 'delta_log-sigma-1-0-mm-3D_glszm_ZonePercentage',
 'delta_log-sigma-1-5-mm-3D_firstorder_Energy',
 'delta_log-sigma-1-5-mm-3D_firstorder_TotalEnergy',
 'delta_log-sigma-1-5-mm-3D_gldm_DependenceVariance',
 'delta_log-sigma-2-0-mm-3D_ngtdm_Busyness',
 'delta_log-sigma-2-5-mm-3D_glszm_SizeZoneNonUniformity',
 'delta_log-sigma-2-5-mm-3D_glszm_ZoneEntropy',
 'delta_logarithm_glszm_LargeAreaHighGrayLevelEmphasis',
 'delta_original_glszm_ZoneVariance',
 'delta_original_shape_LeastAxisLength',
 'delta_square_glcm_Correlation',
 'delta_square_glszm_LowGrayLevelZoneEmphasis',
 'delta_square_glszm_ZonePercentage',
 'delta_wavelet-HHH_glszm_ZoneEntropy',
 'delta_wavelet-HHL_firstorder_Median',
 'delta_wavelet-HHL_firstorder_Uniformity',
 'delta_wavelet-HLH_glcm_ClusterShade',
 'delta_wavelet-HLH_glcm_MaximumProbability',
 'delta_w

In [ ]:
clinical_train[rad_features_list] = final_data[rad_features_list]
clinical_test[rad_features_list] = final_data[rad_features_list]
clinical_train

,encoded_PDL1,delta_log-sigma-1-0-mm-3D_gldm_DependenceVariance,delta_log-sigma-1-0-mm-3D_glszm_LargeAreaLowGrayLevelEmphasis,delta_log-sigma-1-0-mm-3D_glszm_ZoneEntropy,delta_log-sigma-1-0-mm-3D_glszm_ZonePercentage,delta_log-sigma-1-5-mm-3D_firstorder_Energy,delta_log-sigma-1-5-mm-3D_firstorder_TotalEnergy,delta_log-sigma-1-5-mm-3D_gldm_DependenceVariance,delta_log-sigma-2-0-mm-3D_ngtdm_Busyness,delta_log-sigma-2-5-mm-3D_glszm_SizeZoneNonUniformity,...,delta_wavelet-HLL_firstorder_TotalEnergy,delta_wavelet-LHH_glcm_Autocorrelation,delta_wavelet-LHH_glcm_JointEntropy,delta_wavelet-LHH_glrlm_GrayLevelNonUniformityNormalized,delta_wavelet-LHH_glszm_HighGrayLevelZoneEmphasis,delta_wavelet-LHH_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LHL_glszm_SizeZoneNonUniformityNormalized,delta_wavelet-LLH_glszm_GrayLevelNonUniformityNormalized,delta_wavelet-LLH_glszm_SmallAreaEmphasis,delta_wavelet-LLL_glszm_SizeZoneNonUniformityNormalized
pt_no,,,,,,,,,,,,,,,,,,,,,
51127213,1.0,-1.736326,1.587632e+06,0.708757,-0.001276,352.414934,352.414934,-3.233727,934.808585,0.027778,...,509.711550,-0.007470,0.005971,-1.828777e-05,1.238095,0.010580,-0.004244,-0.104376,0.012771,0.001135
38924846,0.0,2.708299,2.344760e+06,1.463922,-0.000981,24.702668,24.702668,-3.321753,-72.336189,0.133333,...,-288.284702,-0.059519,-0.006989,-1.879795e-05,-0.900000,0.450000,0.045444,0.214286,-0.074180,0.010441
56241796,1.0,-1.960175,1.128364e+07,1.213086,-0.002238,341.755395,341.755395,-2.407629,16630.490889,3.250000,...,418.441321,0.014821,0.004977,-6.595540e-06,0.200000,0.249400,0.038792,-0.027874,-0.226132,-0.130211
40815882,1.0,-0.229143,2.104838e+06,-1.437093,-0.001550,8.447886,8.447886,-2.574383,1809.061879,1.666667,...,31.474672,-0.043059,-0.003146,-3.054171e-06,1.050000,-0.200000,0.300000,-0.045351,0.190010,-0.052852
33908849,0.0,-0.363127,5.959548e+06,0.266973,-0.000802,-649.385088,-649.385088,0.166396,16543.918013,-0.055556,...,-777.711610,0.017315,-0.012366,-1.685541e-05,-0.214286,0.165799,0.119773,-0.029220,0.154312,-0.039461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47181472,2.0,1.737556,1.628843e+05,0.403221,-0.000181,197.051072,197.051072,0.689060,2648.254807,-0.454545,...,23.327664,0.005043,-0.007047,-1.320149e-07,0.088235,-0.116891,-0.012191,-0.007686,0.088587,0.088412
49087653,1.0,-1.649570,7.802855e+06,-0.218002,-0.001066,-52.283154,-52.283154,-2.130020,11362.126125,-1.200000,...,508.588955,-0.002807,-0.000420,-5.348455e-05,0.589286,-0.045373,-0.236584,0.064140,-0.135015,-0.022400
49588132,0.0,1.077007,-1.925476e+06,0.717277,0.000503,385.856911,385.856911,0.903566,129382.118764,3.908571,...,431.863021,-0.003426,0.000383,-8.395702e-06,0.200000,-0.046914,0.098765,0.002959,-0.076182,0.092368


In [ ]:
train_data = clinical_train
test_data = clinical_test

In [ ]:
train_data

,encoded_PDL1,delta_log-sigma-1-0-mm-3D_gldm_DependenceVariance,delta_log-sigma-1-0-mm-3D_glszm_LargeAreaLowGrayLevelEmphasis,delta_log-sigma-1-0-mm-3D_glszm_ZoneEntropy,delta_log-sigma-1-0-mm-3D_glszm_ZonePercentage,delta_log-sigma-1-5-mm-3D_firstorder_Energy,delta_log-sigma-1-5-mm-3D_firstorder_TotalEnergy,delta_log-sigma-1-5-mm-3D_gldm_DependenceVariance,delta_log-sigma-2-0-mm-3D_ngtdm_Busyness,delta_log-sigma-2-5-mm-3D_glszm_SizeZoneNonUniformity,...,delta_wavelet-HLL_firstorder_TotalEnergy,delta_wavelet-LHH_glcm_Autocorrelation,delta_wavelet-LHH_glcm_JointEntropy,delta_wavelet-LHH_glrlm_GrayLevelNonUniformityNormalized,delta_wavelet-LHH_glszm_HighGrayLevelZoneEmphasis,delta_wavelet-LHH_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LHL_glszm_SizeZoneNonUniformityNormalized,delta_wavelet-LLH_glszm_GrayLevelNonUniformityNormalized,delta_wavelet-LLH_glszm_SmallAreaEmphasis,delta_wavelet-LLL_glszm_SizeZoneNonUniformityNormalized
pt_no,,,,,,,,,,,,,,,,,,,,,
51127213,1.0,-1.736326,1.587632e+06,0.708757,-0.001276,352.414934,352.414934,-3.233727,934.808585,0.027778,...,509.711550,-0.007470,0.005971,-1.828777e-05,1.238095,0.010580,-0.004244,-0.104376,0.012771,0.001135
38924846,0.0,2.708299,2.344760e+06,1.463922,-0.000981,24.702668,24.702668,-3.321753,-72.336189,0.133333,...,-288.284702,-0.059519,-0.006989,-1.879795e-05,-0.900000,0.450000,0.045444,0.214286,-0.074180,0.010441
56241796,1.0,-1.960175,1.128364e+07,1.213086,-0.002238,341.755395,341.755395,-2.407629,16630.490889,3.250000,...,418.441321,0.014821,0.004977,-6.595540e-06,0.200000,0.249400,0.038792,-0.027874,-0.226132,-0.130211
40815882,1.0,-0.229143,2.104838e+06,-1.437093,-0.001550,8.447886,8.447886,-2.574383,1809.061879,1.666667,...,31.474672,-0.043059,-0.003146,-3.054171e-06,1.050000,-0.200000,0.300000,-0.045351,0.190010,-0.052852
33908849,0.0,-0.363127,5.959548e+06,0.266973,-0.000802,-649.385088,-649.385088,0.166396,16543.918013,-0.055556,...,-777.711610,0.017315,-0.012366,-1.685541e-05,-0.214286,0.165799,0.119773,-0.029220,0.154312,-0.039461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47181472,2.0,1.737556,1.628843e+05,0.403221,-0.000181,197.051072,197.051072,0.689060,2648.254807,-0.454545,...,23.327664,0.005043,-0.007047,-1.320149e-07,0.088235,-0.116891,-0.012191,-0.007686,0.088587,0.088412
49087653,1.0,-1.649570,7.802855e+06,-0.218002,-0.001066,-52.283154,-52.283154,-2.130020,11362.126125,-1.200000,...,508.588955,-0.002807,-0.000420,-5.348455e-05,0.589286,-0.045373,-0.236584,0.064140,-0.135015,-0.022400
49588132,0.0,1.077007,-1.925476e+06,0.717277,0.000503,385.856911,385.856911,0.903566,129382.118764,3.908571,...,431.863021,-0.003426,0.000383,-8.395702e-06,0.200000,-0.046914,0.098765,0.002959,-0.076182,0.092368


In [ ]:
print('Data for Modeling: ' + str(train_data.shape))
print('Unseen Data For Predictions: ' + str(test_data.shape))

Data for Modeling: (79, 35)
Unseen Data For Predictions: (22, 35)


In [ ]:
ypCR_clf = setup(data=train_data,
            test_data=test_data,
            target='encoded_PDL1',
            normalize=True, 
            normalize_method='zscore', 
            data_split_shuffle=True, 
            data_split_stratify='encoded_PDL1',
            fold = 5,
            fold_strategy = 'stratifiedkfold',
            fold_shuffle = True, 
            session_id=3457, 
            use_gpu=True)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) UHD Graphics 750, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that

,Description,Value
0,Session id,3457
1,Target,encoded_PDL1
2,Target type,Multiclass
3,Original data shape,"(101, 35)"
4,Transformed data shape,"(101, 35)"
5,Transformed train set shape,"(79, 35)"
6,Transformed test set shape,"(22, 35)"
7,Numeric features,34
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) UHD Graphics 750, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that

In [ ]:
train = get_config(variable="train_transformed")
train['encoded_PDL1'].value_counts()

1.0    36
0.0    22
2.0    21
Name: encoded_PDL1, dtype: int64

In [ ]:
test = get_config(variable="test_transformed")
test['encoded_PDL1'].value_counts()

1.0    10
2.0     6
0.0     6
Name: encoded_PDL1, dtype: int64

In [ ]:
compare_models(n_select=3, sort='AUC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.6700,0.7971,0.6700,0.6920,0.6720,0.4851,0.4929,0.0240
lr,Logistic Regression,0.6317,0.7754,0.6317,0.6444,0.6128,0.4111,0.4242,0.0480
ada,Ada Boost Classifier,0.5442,0.7061,0.5442,0.5173,0.4881,0.2162,0.2597,0.0860
et,Extra Trees Classifier,0.5050,0.6889,0.5050,0.4809,0.4677,0.1706,0.1647,0.2100
knn,K Neighbors Classifier,0.5450,0.6860,0.5450,0.5842,0.5174,0.2335,0.2518,0.0560
rf,Random Forest Classifier,0.5067,0.6731,0.5067,0.5412,0.4794,0.1896,0.2143,0.2080
gbc,Gradient Boosting Classifier,0.5192,0.6245,0.5192,0.5067,0.4938,0.2314,0.2613,0.3720
nb,Naive Bayes,0.4950,0.6154,0.4950,0.5328,0.4798,0.1835,0.1908,0.0260
lightgbm,Light Gradient Boosting Machine,0.4950,0.6048,0.4950,0.5130,0.4951,0.1968,0.2020,0.5440
qda,Quadratic Discriminant Analysis,0.3792,0.5693,0.3792,0.3872,0.3079,0.1337,0.1693,0.0260


[LinearDiscriminantAnalysis(covariance_estimator=None, n_components=None,
                            priors=None, shrinkage=None, solver='svd',
                            store_covariance=False, tol=0.0001),
 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=3457, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 AdaBoostClassifier(algorithm='SAMME.R', base_estimator='deprecated',
                    estimator=None, learning_rate=1.0, n_estimators=50,
                    random_state=3457)]

In [ ]:
lightgbm = create_model('lightgbm', fold=5, round=4, cross_validation=True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.1250,0.2839,0.1250,0.1250,0.1250,-0.4000,-0.4000
1,0.6250,0.7940,0.6250,0.6806,0.6328,0.4074,0.4183
2,0.4375,0.6987,0.4375,0.4557,0.4429,0.1166,0.1173
3,0.6875,0.7307,0.6875,0.7109,0.6944,0.5122,0.5154
4,0.6000,0.5167,0.6000,0.5926,0.5806,0.3478,0.3592
Mean,0.4950,0.6048,0.4950,0.5130,0.4951,0.1968,0.2020
Std,0.2026,0.1851,0.2026,0.2133,0.2028,0.3253,0.3284


In [ ]:
prediction = predict_model(lightgbm, data=test_data)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.6364,0.6525,0.6364,0.6288,0.6256,0.4211,0.4270
